In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame

Wiki_URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
dfs = pd.read_html(Wiki_URL, header=0)

df = dfs[0]

df["Borough"].replace({"Not assigned": np.nan}, inplace=True)

df.dropna(subset = ["Borough"], inplace=True)
df.reset_index(drop=True, inplace = True)

url = "https://cocl.us/Geospatial_data"
latlng_data = pd.read_csv(url)

merged_df = df.merge(latlng_data, how= 'inner')

merged_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h3>We split the neighborhood column, because in the labs, 1 borough was assigned only 1 neighborhood, so i will follow that format. </h3>

In [2]:
splitdata = DataFrame(merged_df['Neighborhood'].str.split(',').tolist(),index=merged_df['Postal Code']).stack().to_frame()
splitdata = splitdata.reset_index()
splitdata.drop('level_1', axis = 1, inplace = True)
splitdata.head(10)

,Postal Code,0
0,M3A,Parkwoods
1,M4A,Victoria Village
2,M5A,Regent Park
3,M5A,Harbourfront
4,M6A,Lawrence Manor
5,M6A,Lawrence Heights
6,M7A,Queen's Park
7,M7A,Ontario Provincial Government
8,M9A,Islington Avenue
9,M1B,Malvern


In [3]:
merged_df.drop('Neighborhood', axis = 1, inplace = True)
merged_df.head()

,Postal Code,Borough,Latitude,Longitude
0,M3A,North York,43.753259,-79.329656
1,M4A,North York,43.725882,-79.315572
2,M5A,Downtown Toronto,43.654260,-79.360636
3,M6A,North York,43.718518,-79.464763
4,M7A,Downtown Toronto,43.662301,-79.389494


In [4]:
merged_df = merged_df.merge(splitdata, how = "inner")
merged_df.head()

,Postal Code,Borough,Latitude,Longitude,0
0,M3A,North York,43.753259,-79.329656,Parkwoods
1,M4A,North York,43.725882,-79.315572,Victoria Village
2,M5A,Downtown Toronto,43.654260,-79.360636,Regent Park
3,M5A,Downtown Toronto,43.654260,-79.360636,Harbourfront
4,M6A,North York,43.718518,-79.464763,Lawrence Manor


In [5]:
merged_df.rename(columns = {0: 'Neighborhood'}, inplace = True)

In [6]:
merged_df = merged_df[["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"]]
merged_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(merged_df['Borough'].unique()),
        merged_df.shape[0]
    )
)

The dataframe has 10 boroughs and 209 neighborhoods.


In [8]:
toronto = merged_df[merged_df['Borough']=='Downtown Toronto'].reset_index(drop = True)
toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
1,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
2,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
3,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494
4,M5B,Downtown Toronto,Garden District,43.657162,-79.378937


In [9]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [10]:
!pip -q install folium
import folium # map rendering library

In [11]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [12]:
neighborhood = toronto[['Neighborhood',"Latitude","Longitude"]]

In [13]:
neighborhood.head()

,Neighborhood,Latitude,Longitude
0,Regent Park,43.654260,-79.360636
1,Harbourfront,43.654260,-79.360636
2,Queen's Park,43.662301,-79.389494
3,Ontario Provincial Government,43.662301,-79.389494
4,Garden District,43.657162,-79.378937


<h3> Plotting our neighborhoods on a map centered at Toronto </h3>

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(neighborhood['Latitude'], neighborhood['Longitude'], neighborhood['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#ffff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
CLIENT_ID = 'TSRDBNUPONIYCQX55TQ1QSI1GZD4VY5NT3PGZABJ01XGGVMM' # your Foursquare ID
CLIENT_SECRET = 'R4AGKSPNYFCDZOKUXOUIWZ2XA4KQP0XYYUOBCKL1ITSNCORP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TSRDBNUPONIYCQX55TQ1QSI1GZD4VY5NT3PGZABJ01XGGVMM
CLIENT_SECRET:R4AGKSPNYFCDZOKUXOUIWZ2XA4KQP0XYYUOBCKL1ITSNCORP


<h3>Function for getting all venues nearby every neighborhood in Toronto(within 500m)</h3>

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        #Getting all venues nearby current location
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        #retrieving the result
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(neighborhood['Neighborhood'], neighborhood['Latitude'], neighborhood['Longitude'])
print(toronto_venues.shape)

Regent Park
 Harbourfront
Queen's Park
 Ontario Provincial Government
Garden District
 Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond
 Adelaide
 King
Harbourfront East
 Union Station
 Toronto Islands
Toronto Dominion Centre
 Design Exchange
Commerce Court
 Victoria Hotel
University of Toronto
 Harbord
Kensington Market
 Chinatown
 Grange Park
CN Tower
 King and Spadina
 Railway Lands
 Harbourfront West
 Bathurst Quay
 South Niagara
 Island airport
Rosedale
Stn A PO Boxes
St. James Town
 Cabbagetown
First Canadian Place
 Underground city
Church and Wellesley
(2363, 7)


In [44]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,Regent Park,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 204 uniques categories.


<h3>We use one hot encoding to get the frequency of venues per neighborhood</h3>

In [71]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'], prefix="", prefix_sep="")

toronto_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
# toronto_onehot.drop('Neighborhood', axis = 1, inplace = True)

toronto_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'])
toronto_onehot.head()


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
toronto_onehot.shape

(2363, 204)

In [79]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Adelaide,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021505,0.0,...,0.010753,0.0,0.0,0.00,0.010753,0.0,0.000000,0.000000,0.010753,0.0
1,Bathurst Quay,0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,...,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0
2,Cabbagetown,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,Chinatown,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.00,0.035714,0.0,0.053571,0.017857,0.000000,0.0
4,Design Exchange,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.0,...,0.010000,0.0,0.0,0.01,0.010000,0.0,0.000000,0.010000,0.000000,0.0


In [80]:
toronto_grouped.shape

(38, 204)

In [81]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [83]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Gym,Hotel,Clothing Store,Thai Restaurant,Deli / Bodega,Sushi Restaurant,Bookstore
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Coffee Shop,Sculpture Garden
2,Cabbagetown,Coffee Shop,Bakery,Chinese Restaurant,Italian Restaurant,Park,Restaurant,Pizza Place,Café,Pub,Deli / Bodega
3,Chinatown,Café,Dessert Shop,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Vegetarian / Vegan Restaurant,Gaming Cafe,Bar,Burger Joint
4,Design Exchange,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Salad Place,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Concert Hall


In [84]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 4, 0, 4, 4, 0, 1, 1], dtype=int32)

<h3>Merging with our old data which had the lat lng for each neighborhood </h3>

In [85]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Event Space,Performing Arts Venue
1,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Event Space,Performing Arts Venue
2,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0,Coffee Shop,College Cafeteria,Sushi Restaurant,Yoga Studio,Smoothie Shop,Beer Bar,Sandwich Place,Burrito Place,Café,College Auditorium
3,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494,0,Coffee Shop,College Cafeteria,Sushi Restaurant,Yoga Studio,Smoothie Shop,Beer Bar,Sandwich Place,Burrito Place,Café,College Auditorium
4,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Restaurant,Tea Room


<h3> Plotting </h3>

In [90]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, postal, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Postal Code'] ,toronto_merged['Cluster Labels']):
    label = folium.Popup('Code - ' + str(postal)+'\nName - '+str(poi) + ' Cluster - ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2> Examine Clusters </h2>

<h2>Cluster 1</h2>

In [93]:
#cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Regent Park,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Event Space,Performing Arts Venue
1,Harbourfront,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Event Space,Performing Arts Venue
2,Queen's Park,0,Coffee Shop,College Cafeteria,Sushi Restaurant,Yoga Studio,Smoothie Shop,Beer Bar,Sandwich Place,Burrito Place,Café,College Auditorium
3,Ontario Provincial Government,0,Coffee Shop,College Cafeteria,Sushi Restaurant,Yoga Studio,Smoothie Shop,Beer Bar,Sandwich Place,Burrito Place,Café,College Auditorium
4,Garden District,0,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Restaurant,Tea Room
5,Ryerson,0,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Restaurant,Tea Room
6,St. James Town,0,Coffee Shop,Café,Italian Restaurant,Gastropub,Bakery,Cocktail Bar,Restaurant,American Restaurant,Park,Chinese Restaurant
7,Berczy Park,0,Coffee Shop,Cocktail Bar,Café,Bakery,Seafood Restaurant,Beer Bar,Restaurant,Cheese Shop,Bistro,Concert Hall
8,Central Bay Street,0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Japanese Restaurant,Thai Restaurant,Burger Joint,Bubble Tea Shop,Bar
10,Richmond,0,Coffee Shop,Café,Restaurant,Gym,Hotel,Clothing Store,Thai Restaurant,Deli / Bodega,Sushi Restaurant,Bookstore


<h2>Cluster 2</h2>

In [94]:
#cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,CN Tower,1,Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Coffee Shop,Sculpture Garden
26,King and Spadina,1,Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Coffee Shop,Sculpture Garden
27,Railway Lands,1,Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Coffee Shop,Sculpture Garden
28,Harbourfront West,1,Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Coffee Shop,Sculpture Garden
29,Bathurst Quay,1,Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Coffee Shop,Sculpture Garden
30,South Niagara,1,Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Coffee Shop,Sculpture Garden
31,Island airport,1,Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Coffee Shop,Sculpture Garden


<h2>Cluster 3</h2>

In [95]:
#cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Rosedale,2,Park,Playground,Trail,Dance Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


<h2>Cluster 4</h2>

In [97]:
#cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Christie,3,Grocery Store,Café,Park,Restaurant,Coffee Shop,Nightclub,Diner,Candy Store,Italian Restaurant,Baby Store


<h2>Cluster 5</h2>

In [101]:
#cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,University of Toronto,4,Café,Bookstore,Bar,Italian Restaurant,Japanese Restaurant,Bakery,Restaurant,Yoga Studio,Sandwich Place,Beer Bar
21,Harbord,4,Café,Bookstore,Bar,Italian Restaurant,Japanese Restaurant,Bakery,Restaurant,Yoga Studio,Sandwich Place,Beer Bar
22,Kensington Market,4,Café,Dessert Shop,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Vegetarian / Vegan Restaurant,Gaming Cafe,Bar,Burger Joint
23,Chinatown,4,Café,Dessert Shop,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Vegetarian / Vegan Restaurant,Gaming Cafe,Bar,Burger Joint
24,Grange Park,4,Café,Dessert Shop,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Vegetarian / Vegan Restaurant,Gaming Cafe,Bar,Burger Joint


<h3>Most common venues:</h3>

<h3>Cluster1 - Clothing and Coffee Shop</h3>
<h3>Cluster2 - Airport</h3>
<h3>Cluster3 - Park</h3>
<h3>Cluster4 - Grocery Stores</h3>
<h3>Cluster5 - Cafe</h3>

